# Obstacle Avoidance

## Aseba code

var speed0 = 100       # nominal speed  
var speedGain = 2      # gain used with ground gradient  
var obstThrL = 10      # low obstacle threshold to switch state 1->0  
var obstThrH = 20      # high obstacle threshold to switch state 0->1  
var obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)  
  
var state = 0          # 0=gradient, 1=obstacle avoidance  
var diffDelta          # difference between ground sensors  
var obst[2]            # measurements from left and right prox sensors  
  
timer.period[0] = 10   # 10ms sampling time  
  
onevent timer0  
  .# acquisition from ground sensor for going toward the goal  
  diffDelta = prox.ground.delta[1] - prox.ground.delta[0]  
  .# acquisition from the proximity sensors to detect obstacles  
  obst = [prox.horizontal[0], prox.horizontal[4]]  
  if state == 0 and (obst[0] > obstThrH or obst[1] > obstThrH) then  
    # switch from goal tracking to obst avoidance if obstacle detected  
    state = 1  
  elseif state == 1 and obst[0] < obstThrL and obst[1] < obstThrL then  
    # switch from obst avoidance to goal tracking if obstacle got unseen  
    state = 0  
  end  
  if  state == 0 then  
    # goal tracking: turn toward the goal  
    motor.left.target = speed0 - speedGain * diffDelta  
    motor.right.target = speed0 + speedGain * diffDelta  
  else  
    # obstacle avoidance: accelerate wheel near obstacle  
    motor.left.target = speed0 + obstSpeedGain * (obst[0] / 100)  
    motor.right.target = speed0 + obstSpeedGain * (obst[1] / 100)  
  end  

## Python code

### Imports

In [7]:
import os
import sys
import time
import serial
#import math
import numpy as np
#from scipy.interpolate import interp1d

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
#from local_occupancy import sensor_measurements, sensor_distances
#from local_occupancy import thymio_coords, sensor_pos_from_center, sensor_angles

In [9]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

### Accessible functions for the robot

In [10]:
dir(robot)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'auto_handshake',
 'close',
 'get_node_description',
 'get_target_node_id',
 'get_target_node_var_total_size',
 'get_var',
 'get_var_array',
 'get_variables',
 'handle_message',
 'handshake',
 'input_lock',
 'input_thread',
 'io',
 'list_nodes',
 'node_id',
 'null',
 'output_lock',
 'refresh_thread',
 'refreshing_timeout',
 'refreshing_trigger',
 'remote_node',
 'send',
 'serial',
 'serial_default_port',
 'set_refreshing_rate',
 'set_var',
 'set_var_array',
 'set_variables',
 'tcp',
 'terminating',
 'variable_description']

### Read/Write variables

In [11]:
variables = robot.variable_description()
for var in variables :
    print(var)

{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'prox.ground.ambiant', 'offset': 80, 'size': 2}
{'name': 'prox.ground.refl

### Variables globales

In [12]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.2 #in seconds
TIME_SLEEP_DATA = 0.2 #5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4

### Main function 

In [18]:
for i in range(1000000): #simulation of big while loop
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
        
    time.sleep(TIME_SLEEP_DATA)
robot.set_var("motor.left.target", MOTOR_SPEED)   #TO REMOVE
robot.set_var("motor.right.target", MOTOR_SPEED)  #TO REMOVE

KeyboardInterrupt: 

### Tests

In [14]:
for i in range(1000000): #simulation of big while loop
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    print(sensor_measurment)                               #TO REMOVE
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
        #break ?
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            print(sensor_measurment)                      #TO REMOVE
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
        
    time.sleep(TIME_SLEEP_DATA)
robot.set_var("motor.left.target", MOTOR_SPEED)   #TO REMOVE
robot.set_var("motor.right.target", MOTOR_SPEED)  #TO REMOVE

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 1418, 0, 0, 0, 0]
[0, 0, 1689, 0, 0, 0, 0]
[0, 0, 1820, 0, 0, 0, 0]
[0, 0, 2077, 0, 0, 0, 0]
[0, 0, 2339, 0, 0, 0, 0]
[0, 0, 2799, 0, 0, 0, 0]
[0, 0, 3183, 0, 0, 0, 0]
[0, 0, 3631, 0, 0, 0, 0]
[0, 0, 3873, 0, 0, 0, 0]
[0, 0, 3873, 0, 0, 0, 0]
[0, 0, 4190, 0, 0, 0, 0]
[0, 0, 3926, 2776, 0, 0, 0]
[0, 0, 2381, 4334, 0, 0, 0]
[0, 0, 0, 4435, 0, 0, 0]
[0, 0, 0, 3302, 4082, 0, 0]
[0, 0, 0, 0, 4207, 0, 0]
[0, 0, 0, 0, 4183, 0, 0]
[0, 0, 0, 0, 2576, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 

KeyboardInterrupt: 

In [19]:
sensor_measurment = robot["prox.horizontal"]
obstacle_direction = np.nonzero(sensor_measurment)[0]

print(sensor_measurment)
print(obstacle_direction)
#print(np.argmax(sensor_measurment))
print(np.shape(obstacle_direction)[0])
print(np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_5)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_6)))
print(np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)))

#robot.set_var("motor.left.target", 50)
#robot.set_var("motor.right.target", 2**16-50)
#time.sleep(1.1)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)

[0, 0, 1406, 1788, 0, 0, 0]
[2 3]
2
False
True
True
True


### Bugs to fix

1) if the robot meet a v shape obstacle  
2) if the robot has an obstacle in front of sensor n°2 it can turns and then sense the obstacle on the 3th, it turns backwards --> wait a bit more in the case of detection on the sensor 2 ?